In [1]:
!pip install datasets transformers ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import ipywidgets as widgets
from IPython.display import display

In [3]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

In [4]:
ag_news = load_dataset("ag_news")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_len = 128

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_len)

tokenized_ag_news = ag_news.map(preprocess_function, batched=True)

class AGNewsDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
            "label": torch.tensor(item["label"], dtype=torch.long)
        }

train_dataset = AGNewsDataset(tokenized_ag_news["train"])
test_dataset = AGNewsDataset(tokenized_ag_news["test"])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
vocab_size = tokenizer.vocab_size
embed_size = 128
hidden_size = 128
output_size = 4

model = RNNModel(vocab_size, embed_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"]
            labels = batch["label"]

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Save the trained model
#torch.save(model.state_dict(), "rnn_model.pth")

# To load the model later
# model.load_state_dict(torch.load("rnn_model.pth"))
# model.eval()


In [7]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"]
            labels = batch["label"]

            outputs = model(input_ids)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

In [8]:
train_model(model, train_dataloader, criterion, optimizer, num_epochs=5)
torch.save(model.state_dict(), "rnn_model.pth")
evaluate_model(model, test_dataloader)

Epoch 1/5, Loss: 0.8293
Epoch 2/5, Loss: 0.2763
Epoch 3/5, Loss: 0.1991
Epoch 4/5, Loss: 0.1496
Epoch 5/5, Loss: 0.1119
Accuracy: 91.39%


In [9]:
# Function to predict using the RNN model
def predict_news_article(text):
    tokenized_input = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_len)
    input_ids = tokenized_input['input_ids']

    model.eval()
    with torch.no_grad():
        output = model(input_ids)
        prediction = torch.argmax(output, dim=1).item()
    return prediction

# Define a simple function to process the user input
def on_button_click(b):
    text = text_box.value
    prediction = predict_news_article(text)
    result.value = f"Prediction: {prediction}"

# Create the text input box
text_box = widgets.Textarea(
    placeholder='Type a news article here...',
    description='Input Text:',
    disabled=False,
    layout=widgets.Layout(width='50%', height='100px')
)

# Create a button to submit the text
submit_button = widgets.Button(
    description='Predict',
    disabled=False,
    button_style='success',
    tooltip='Click to get prediction',
    icon='check'
)

# Create an output box to display the result
result = widgets.Label(value='')

# Link the button click event to the prediction function
submit_button.on_click(on_button_click)

# Display the widgets
display(text_box, submit_button, result)


Textarea(value='', description='Input Text:', layout=Layout(height='100px', width='50%'), placeholder='Type a …

Button(button_style='success', description='Predict', icon='check', style=ButtonStyle(), tooltip='Click to get…

Label(value='')